## Libs

In [1]:
%pip install pandas numpy scipy plotly regex ipywidgets tqdm -q


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [161]:
import os
import re

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from tqdm.auto import tqdm

# My modules
import parsing
import posing
import plotting

## Retrieving Animations

In [176]:
def get_animation_df(asf_file, amc_file):

    asf_dict = parsing.parse_asf(asf_file)
    amc_dict = parsing.parse_amc(amc_file)

    rest_df = parsing.get_rest_df(asf_dict)
    motion_df = parsing.get_motion_df(amc_dict)

    animation_df = pd.concat([posing.apply_pose(rest_df, motion_df[i]) for i in tqdm(motion_df.columns)], keys=motion_df.columns)
    animation_df.index.names = ['frame', 'joint']
    
    return animation_df

animations = {
    'Hopping': get_animation_df('../data/subjects_small/01/01.asf', '../data/subjects_small/01/01_01.amc'),
    'Walking (Flat Terrain)': get_animation_df('../data/subjects_small/02/02.asf', '../data/subjects_small/02/02_01.amc'),
    'Walking (Uneven Terrain)': get_animation_df('../data/subjects_small/03/03.asf', '../data/subjects_small/03/03_01.amc')
}

  0%|          | 0/2751 [00:00<?, ?it/s]

  0%|          | 0/343 [00:00<?, ?it/s]

  0%|          | 0/432 [00:00<?, ?it/s]

## Torso Verticality

In [92]:
def get_torso_verticality(animation_df):
    torso_vert = animation_df.loc[:, 'lowerneck', :]['position'] - animation_df.loc[:, 'lowerback', :]['position']
    torso_vert /= torso_vert.apply(np.linalg.norm)
    torso_vert = torso_vert.apply(lambda x: np.dot(np.array([0, 1, 0]), x))
    return torso_vert

In [177]:
for name, df in animations.items():
    torso_vert = get_torso_verticality(df)
    display(px.line(torso_vert, title=name+' Torso Verticality', labels={'x':'Frame', 'y':'Torso Verticality'}))

## Jerk Magnitude

In [150]:
def get_diff(animation_df):

    joints = animation_df.loc[0].index
    final_df = pd.DataFrame()
    
    for j in joints:
        j_df = animation_df.loc[(slice(None), j), :]
        new_df = pd.DataFrame(index = j_df.index)
        new_df['velocity'] = j_df['position'].diff() * 120.0
        new_df['acceleration'] = new_df['velocity'].diff() * 120.0
        new_df['jerk'] = new_df['acceleration'].diff() * 120.0

        final_df = pd.concat([final_df, new_df])

    return final_df.reindex(animation_df.index)

In [179]:
for name, df in animations.items():
    diff = get_diff(df)
    jerk = diff['jerk'].apply(np.linalg.norm)
    display(px.line(jerk.loc[:, 'root'], title=name+' Jerk Magnitude'))